# 6.3.1 自定義算法實作挑戰 - 高級練習

本練習挑戰你從零開始實現經典的電腦視覺算法，深入理解算法原理並掌握優化技巧。

## 練習目標
- 從理論到實作：深入理解算法數學基礎
- 性能優化：實現接近OpenCV官方實現的效能
- 代碼品質：撰寫可維護、可擴展的高質量代碼
- 測試驗證：建立完整的測試和驗證流程
- 創新應用：在經典算法基礎上進行創新改進

## 難度等級: ⭐⭐⭐⭐⭐ (專家級)

## 環境設置與導入

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
import math
from typing import List, Tuple, Optional, Union
from numba import jit, prange
import warnings
warnings.filterwarnings('ignore')

# 添加utils路徑
sys.path.append('../../utils')
from image_utils import load_image, resize_image
from visualization import display_image, display_multiple_images
from performance import time_function, benchmark_function

# 設置matplotlib
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 環境設置完成")
print(f"OpenCV版本: {cv2.__version__}")
print(f"NumPy版本: {np.__version__}")
try:
    import numba
    print(f"Numba版本: {numba.__version__} (加速計算可用)")
except:
    print("⚠️ Numba未安裝，某些優化功能不可用")

## 挑戰1: 自定義Canny邊緣檢測算法

### 任務描述
從零實現完整的Canny邊緣檢測算法，包括高斯濾波、梯度計算、非極大值抑制、雙閾值處理和邊緣連接。

In [ ]:
class CustomCannyDetector:
    """
    自定義Canny邊緣檢測器實現
    
    實現步驟：
    1. 高斯模糊去噪
    2. 計算圖像梯度
    3. 非極大值抑制
    4. 雙閾值處理
    5. 邊緣連接
    """
    
    def __init__(self, gaussian_kernel_size=5, gaussian_sigma=1.4):
        """
        初始化Canny檢測器
        
        Args:
            gaussian_kernel_size: 高斯核大小
            gaussian_sigma: 高斯標準差
        """
        self.kernel_size = gaussian_kernel_size
        self.sigma = gaussian_sigma
        self.gaussian_kernel = self._create_gaussian_kernel(gaussian_kernel_size, gaussian_sigma)
        
        # Sobel算子
        self.sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
        self.sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)
    
    def _create_gaussian_kernel(self, size, sigma):
        """
        創建高斯卷積核
        
        Args:
            size: 核大小
            sigma: 標準差
        
        Returns:
            np.array: 高斯核
        """
        kernel = np.zeros((size, size))
        center = size // 2
        
        # 計算高斯權重
        for i in range(size):
            for j in range(size):
                x, y = i - center, j - center
                kernel[i, j] = np.exp(-(x*x + y*y) / (2 * sigma * sigma))
        
        # 正規化
        kernel = kernel / np.sum(kernel)
        return kernel
    
    @jit(nopython=True)
    def _apply_convolution(self, image, kernel):
        """
        應用卷積操作（Numba加速）
        
        Args:
            image: 輸入圖像
            kernel: 卷積核
        
        Returns:
            np.array: 卷積結果
        """
        h, w = image.shape
        kh, kw = kernel.shape
        pad_h, pad_w = kh // 2, kw // 2
        
        result = np.zeros((h, w), dtype=np.float32)
        
        for i in range(pad_h, h - pad_h):
            for j in range(pad_w, w - pad_w):
                conv_sum = 0.0
                for ki in range(kh):
                    for kj in range(kw):
                        conv_sum += image[i - pad_h + ki, j - pad_w + kj] * kernel[ki, kj]
                result[i, j] = conv_sum
        
        return result
    
    def _gaussian_blur(self, image):
        """
        應用高斯模糊
        
        Args:
            image: 輸入圖像
        
        Returns:
            np.array: 模糊後的圖像
        """
        return self._apply_convolution(image.astype(np.float32), self.gaussian_kernel)
    
    def _compute_gradients(self, image):
        """
        計算圖像梯度
        
        Args:
            image: 輸入圖像
        
        Returns:
            tuple: (梯度幅值, 梯度方向)
        """
        # 計算x和y方向的梯度
        grad_x = self._apply_convolution(image, self.sobel_x)
        grad_y = self._apply_convolution(image, self.sobel_y)
        
        # 計算梯度幅值和方向
        gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
        gradient_direction = np.arctan2(grad_y, grad_x)
        
        return gradient_magnitude, gradient_direction
    
    @jit(nopython=True)
    def _non_maximum_suppression(self, magnitude, direction):
        """
        非極大值抑制（Numba加速）
        
        Args:
            magnitude: 梯度幅值
            direction: 梯度方向
        
        Returns:
            np.array: 抑制後的邊緣圖像
        """
        h, w = magnitude.shape
        suppressed = np.zeros((h, w), dtype=np.float32)
        
        for i in range(1, h - 1):
            for j in range(1, w - 1):
                angle = direction[i, j]
                
                # 將角度轉換到0-180度
                angle = angle * 180.0 / np.pi
                if angle < 0:
                    angle += 180
                
                # 根據梯度方向確定相鄰像素
                if (0 <= angle < 22.5) or (157.5 <= angle <= 180):
                    # 水平方向
                    neighbor1 = magnitude[i, j-1]
                    neighbor2 = magnitude[i, j+1]
                elif 22.5 <= angle < 67.5:
                    # 45度方向
                    neighbor1 = magnitude[i-1, j+1]
                    neighbor2 = magnitude[i+1, j-1]
                elif 67.5 <= angle < 112.5:
                    # 垂直方向
                    neighbor1 = magnitude[i-1, j]
                    neighbor2 = magnitude[i+1, j]
                else:  # 112.5 <= angle < 157.5
                    # 135度方向
                    neighbor1 = magnitude[i-1, j-1]
                    neighbor2 = magnitude[i+1, j+1]
                
                # 非極大值抑制
                if magnitude[i, j] >= neighbor1 and magnitude[i, j] >= neighbor2:
                    suppressed[i, j] = magnitude[i, j]
        
        return suppressed
    
    def _double_threshold(self, image, low_threshold, high_threshold):
        """
        雙閾值處理
        
        Args:
            image: 非極大值抑制後的圖像
            low_threshold: 低閾值
            high_threshold: 高閾值
        
        Returns:
            np.array: 分類後的邊緣圖像
        """
        # 創建三種像素類型：強邊緣、弱邊緣、非邊緣
        strong_edges = 255
        weak_edges = 75
        
        result = np.zeros_like(image, dtype=np.uint8)
        
        # 強邊緣
        strong_mask = image >= high_threshold
        result[strong_mask] = strong_edges
        
        # 弱邊緣
        weak_mask = (image >= low_threshold) & (image < high_threshold)
        result[weak_mask] = weak_edges
        
        return result
    
    @jit(nopython=True)
    def _edge_tracking_by_hysteresis(self, image):
        """
        通過滯後閾值進行邊緣連接（Numba加速）
        
        Args:
            image: 雙閾值處理後的圖像
        
        Returns:
            np.array: 最終的邊緣圖像
        """
        h, w = image.shape
        result = image.copy()
        
        # 多次迭代以確保所有連接的弱邊緣都被保留
        for _ in range(10):  # 限制迭代次數避免無限循環
            changed = False
            
            for i in range(1, h - 1):
                for j in range(1, w - 1):
                    if result[i, j] == 75:  # 弱邊緣
                        # 檢查8鄰域是否有強邊緣
                        has_strong_neighbor = False
                        for di in range(-1, 2):
                            for dj in range(-1, 2):
                                if di == 0 and dj == 0:
                                    continue
                                if result[i + di, j + dj] == 255:
                                    has_strong_neighbor = True
                                    break
                            if has_strong_neighbor:
                                break
                        
                        if has_strong_neighbor:
                            result[i, j] = 255  # 提升為強邊緣
                            changed = True
            
            if not changed:
                break
        
        # 移除剩餘的弱邊緣
        result[result == 75] = 0
        
        return result
    
    def detect_edges(self, image, low_threshold=50, high_threshold=150):
        """
        執行完整的Canny邊緣檢測
        
        Args:
            image: 輸入圖像
            low_threshold: 低閾值
            high_threshold: 高閾值
        
        Returns:
            dict: 包含所有中間結果的字典
        """
        # 轉換為灰階
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        print("🔄 執行自定義Canny邊緣檢測...")
        
        # 步驟1: 高斯模糊
        print("  1. 高斯模糊去噪...")
        start_time = time.time()
        blurred = self._gaussian_blur(gray)
        blur_time = (time.time() - start_time) * 1000
        
        # 步驟2: 計算梯度
        print("  2. 計算圖像梯度...")
        start_time = time.time()
        magnitude, direction = self._compute_gradients(blurred)
        gradient_time = (time.time() - start_time) * 1000
        
        # 步驟3: 非極大值抑制
        print("  3. 非極大值抑制...")
        start_time = time.time()
        suppressed = self._non_maximum_suppression(magnitude, direction)
        nms_time = (time.time() - start_time) * 1000
        
        # 步驟4: 雙閾值處理
        print("  4. 雙閾值處理...")
        start_time = time.time()
        thresholded = self._double_threshold(suppressed, low_threshold, high_threshold)
        threshold_time = (time.time() - start_time) * 1000
        
        # 步驟5: 邊緣連接
        print("  5. 邊緣連接...")
        start_time = time.time()
        final_edges = self._edge_tracking_by_hysteresis(thresholded)
        hysteresis_time = (time.time() - start_time) * 1000
        
        total_time = blur_time + gradient_time + nms_time + threshold_time + hysteresis_time
        
        print(f"  ✅ 完成！總時間: {total_time:.1f}ms")
        
        return {
            'original': gray,
            'blurred': blurred.astype(np.uint8),
            'magnitude': (magnitude / np.max(magnitude) * 255).astype(np.uint8),
            'suppressed': (suppressed / np.max(suppressed) * 255).astype(np.uint8) if np.max(suppressed) > 0 else suppressed.astype(np.uint8),
            'thresholded': thresholded,
            'final_edges': final_edges,
            'processing_times': {
                'gaussian_blur': blur_time,
                'gradient_computation': gradient_time,
                'non_maximum_suppression': nms_time,
                'double_threshold': threshold_time,
                'edge_tracking': hysteresis_time,
                'total': total_time
            }
        }

# 創建自定義Canny檢測器
custom_canny = CustomCannyDetector()
print("✅ 自定義Canny邊緣檢測器初始化完成")

### 🎯 挑戰任務 1.1: 測試自定義Canny算法

**任務**: 使用自定義實現的Canny算法並與OpenCV官方實現進行比較。

In [ ]:
def compare_canny_implementations(image_path):
    """
    比較自定義Canny實現與OpenCV官方實現
    
    Args:
        image_path: 測試圖像路徑
    """
    if not os.path.exists(image_path):
        print(f"圖像不存在: {image_path}")
        return
    
    # 載入圖像
    image = load_image(image_path)
    if image is None:
        return
    
    # 調整圖像大小（避免計算時間過長）
    image_resized = resize_image(image, max_width=400)
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    
    print(f"\n🔍 Canny實現比較測試: {os.path.basename(image_path)}")
    print(f"圖像大小: {gray.shape}")
    print("-" * 50)
    
    # 測試自定義實現
    print("\n🛠️  自定義實現:")
    start_time = time.time()
    custom_result = custom_canny.detect_edges(gray, low_threshold=50, high_threshold=150)
    custom_total_time = (time.time() - start_time) * 1000
    
    # 測試OpenCV實現
    print("\n📚 OpenCV實現:")
    start_time = time.time()
    opencv_edges = cv2.Canny(gray, 50, 150)
    opencv_time = (time.time() - start_time) * 1000
    print(f"  處理時間: {opencv_time:.1f}ms")
    
    # 顯示中間步驟
    intermediate_images = [
        custom_result['original'],
        custom_result['blurred'],
        custom_result['magnitude'],
        custom_result['suppressed']
    ]
    
    intermediate_titles = [
        "原始圖像",
        "高斯模糊",
        "梯度幅值",
        "非極大值抑制"
    ]
    
    print("\n📊 顯示中間處理步驟:")
    display_multiple_images(intermediate_images, intermediate_titles, figsize=(16, 8))
    
    # 顯示最終比較結果
    final_images = [
        custom_result['original'],
        custom_result['final_edges'],
        opencv_edges,
        cv2.absdiff(custom_result['final_edges'], opencv_edges)
    ]
    
    final_titles = [
        "原始圖像",
        f"自定義Canny\n{custom_total_time:.1f}ms",
        f"OpenCV Canny\n{opencv_time:.1f}ms",
        "差異圖像"
    ]
    
    print("\n🏆 最終結果比較:")
    display_multiple_images(final_images, final_titles, figsize=(16, 8))
    
    # 性能分析
    print("\n📈 性能分析:")
    print("-" * 40)
    print(f"自定義實現總時間: {custom_total_time:.1f}ms")
    print(f"OpenCV實現時間:   {opencv_time:.1f}ms")
    print(f"性能比值:         {custom_total_time/opencv_time:.1f}x")
    
    # 詳細時間分解
    print("\n⏱️  自定義實現時間分解:")
    for step, time_ms in custom_result['processing_times'].items():
        if step != 'total':
            percentage = (time_ms / custom_total_time) * 100
            print(f"  {step:20}: {time_ms:6.1f}ms ({percentage:4.1f}%)")
    
    # 質量評估
    print("\n🎯 質量評估:")
    diff_pixels = np.sum(cv2.absdiff(custom_result['final_edges'], opencv_edges) > 0)
    total_pixels = custom_result['final_edges'].shape[0] * custom_result['final_edges'].shape[1]
    similarity = (1 - diff_pixels / total_pixels) * 100
    
    print(f"  相似度:     {similarity:.1f}%")
    print(f"  差異像素:   {diff_pixels:,} / {total_pixels:,}")
    
    if similarity >= 95:
        quality_rating = "優秀 ⭐⭐⭐⭐⭐"
    elif similarity >= 85:
        quality_rating = "良好 ⭐⭐⭐⭐"
    elif similarity >= 70:
        quality_rating = "一般 ⭐⭐⭐"
    else:
        quality_rating = "需改進 ⭐⭐"
    
    print(f"  質量評級:   {quality_rating}")
    
    return custom_result, opencv_edges, {
        'custom_time': custom_total_time,
        'opencv_time': opencv_time,
        'similarity': similarity
    }

# 測試自定義Canny實現
test_images = [
    "../../assets/images/basic/face03.jpg",
    "../../assets/images/basic/faces01.jpg"
]

for image_path in test_images:
    if os.path.exists(image_path):
        custom_result, opencv_result, metrics = compare_canny_implementations(image_path)
        break
else:
    print("⚠️ 沒有找到測試圖像")

## 挑戰2: 自定義Harris角點檢測算法

### 任務描述
實現完整的Harris角點檢測算法，包括結構張量計算、Harris響應函數、非極大值抑制等步驟。

In [ ]:
class CustomHarrisDetector:
    """
    自定義Harris角點檢測器實現
    
    Harris角點檢測步驟：
    1. 計算圖像梯度
    2. 計算結構張量
    3. 計算Harris響應函數
    4. 非極大值抑制
    5. 閾值處理提取角點
    """
    
    def __init__(self, k=0.04, window_size=3, gaussian_sigma=1.0):
        """
        初始化Harris檢測器
        
        Args:
            k: Harris參數
            window_size: 窗口大小
            gaussian_sigma: 高斯標準差
        """
        self.k = k
        self.window_size = window_size
        self.sigma = gaussian_sigma
        
        # 創建高斯權重窗口
        self.gaussian_window = self._create_gaussian_window(window_size, gaussian_sigma)
        
        # Sobel算子
        self.sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
        self.sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)
    
    def _create_gaussian_window(self, size, sigma):
        """
        創建高斯權重窗口
        
        Args:
            size: 窗口大小
            sigma: 標準差
        
        Returns:
            np.array: 高斯權重窗口
        """
        window = np.zeros((size, size))
        center = size // 2
        
        for i in range(size):
            for j in range(size):
                x, y = i - center, j - center
                window[i, j] = np.exp(-(x*x + y*y) / (2 * sigma * sigma))
        
        return window / np.sum(window)
    
    def _compute_gradients(self, image):
        """
        計算圖像梯度
        
        Args:
            image: 輸入圖像
        
        Returns:
            tuple: (Ix, Iy) 梯度圖像
        """
        # 使用Sobel算子計算梯度
        Ix = cv2.filter2D(image.astype(np.float32), -1, self.sobel_x)
        Iy = cv2.filter2D(image.astype(np.float32), -1, self.sobel_y)
        
        return Ix, Iy
    
    @jit(nopython=True)
    def _compute_structure_tensor(self, Ix, Iy, window):
        """
        計算結構張量（Numba加速）
        
        Args:
            Ix, Iy: 梯度圖像
            window: 高斯權重窗口
        
        Returns:
            tuple: (M11, M12, M22) 結構張量元素
        """
        h, w = Ix.shape
        window_size = window.shape[0]
        offset = window_size // 2
        
        M11 = np.zeros((h, w), dtype=np.float32)
        M12 = np.zeros((h, w), dtype=np.float32)
        M22 = np.zeros((h, w), dtype=np.float32)
        
        for i in range(offset, h - offset):
            for j in range(offset, w - offset):
                # 計算局部區域的加權梯度乘積
                sum_IxIx = 0.0
                sum_IxIy = 0.0
                sum_IyIy = 0.0
                
                for ki in range(window_size):
                    for kj in range(window_size):
                        pi, pj = i - offset + ki, j - offset + kj
                        weight = window[ki, kj]
                        
                        ix_val = Ix[pi, pj]
                        iy_val = Iy[pi, pj]
                        
                        sum_IxIx += weight * ix_val * ix_val
                        sum_IxIy += weight * ix_val * iy_val
                        sum_IyIy += weight * iy_val * iy_val
                
                M11[i, j] = sum_IxIx
                M12[i, j] = sum_IxIy
                M22[i, j] = sum_IyIy
        
        return M11, M12, M22
    
    def _compute_harris_response(self, M11, M12, M22):
        """
        計算Harris響應函數
        
        Args:
            M11, M12, M22: 結構張量元素
        
        Returns:
            np.array: Harris響應
        """
        # 計算行列式和跡
        det_M = M11 * M22 - M12 * M12
        trace_M = M11 + M22
        
        # Harris響應函數: R = det(M) - k * trace(M)^2
        R = det_M - self.k * (trace_M ** 2)
        
        return R
    
    @jit(nopython=True)
    def _non_maximum_suppression(self, response, window_size=3):
        """
        非極大值抑制（Numba加速）
        
        Args:
            response: Harris響應圖像
            window_size: 抑制窗口大小
        
        Returns:
            np.array: 抑制後的響應圖像
        """
        h, w = response.shape
        suppressed = np.zeros((h, w), dtype=np.float32)
        offset = window_size // 2
        
        for i in range(offset, h - offset):
            for j in range(offset, w - offset):
                center_val = response[i, j]
                is_maximum = True
                
                # 檢查局部窗口內是否為最大值
                for ki in range(-offset, offset + 1):
                    for kj in range(-offset, offset + 1):
                        if ki == 0 and kj == 0:
                            continue
                        if response[i + ki, j + kj] > center_val:
                            is_maximum = False
                            break
                    if not is_maximum:
                        break
                
                if is_maximum:
                    suppressed[i, j] = center_val
        
        return suppressed
    
    def detect_corners(self, image, threshold=0.01, nms_window=5):
        """
        執行完整的Harris角點檢測
        
        Args:
            image: 輸入圖像
            threshold: 響應閾值
            nms_window: 非極大值抑制窗口大小
        
        Returns:
            dict: 檢測結果和中間數據
        """
        # 轉換為灰階
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        print("🔄 執行自定義Harris角點檢測...")
        
        # 步驟1: 計算梯度
        print("  1. 計算圖像梯度...")
        start_time = time.time()
        Ix, Iy = self._compute_gradients(gray)
        gradient_time = (time.time() - start_time) * 1000
        
        # 步驟2: 計算結構張量
        print("  2. 計算結構張量...")
        start_time = time.time()
        M11, M12, M22 = self._compute_structure_tensor(Ix, Iy, self.gaussian_window)
        tensor_time = (time.time() - start_time) * 1000
        
        # 步驟3: 計算Harris響應
        print("  3. 計算Harris響應...")
        start_time = time.time()
        response = self._compute_harris_response(M11, M12, M22)
        response_time = (time.time() - start_time) * 1000
        
        # 步驟4: 非極大值抑制
        print("  4. 非極大值抑制...")
        start_time = time.time()
        suppressed = self._non_maximum_suppression(response, nms_window)
        nms_time = (time.time() - start_time) * 1000
        
        # 步驟5: 閾值處理提取角點
        print("  5. 提取角點...")
        start_time = time.time()
        
        # 自適應閾值（相對於最大響應值）
        max_response = np.max(suppressed)
        adaptive_threshold = threshold * max_response
        
        # 提取角點座標
        corner_mask = suppressed > adaptive_threshold
        corner_coords = np.where(corner_mask)
        corners = list(zip(corner_coords[1], corner_coords[0]))  # (x, y) 格式
        
        extraction_time = (time.time() - start_time) * 1000
        
        total_time = gradient_time + tensor_time + response_time + nms_time + extraction_time
        
        print(f"  ✅ 完成！檢測到 {len(corners)} 個角點，總時間: {total_time:.1f}ms")
        
        return {
            'original': gray,
            'gradient_x': (np.abs(Ix) / np.max(np.abs(Ix)) * 255).astype(np.uint8),
            'gradient_y': (np.abs(Iy) / np.max(np.abs(Iy)) * 255).astype(np.uint8),
            'response': response,
            'suppressed': suppressed,
            'corners': corners,
            'corner_mask': corner_mask,
            'parameters': {
                'k': self.k,
                'threshold': threshold,
                'adaptive_threshold': adaptive_threshold,
                'max_response': max_response
            },
            'processing_times': {
                'gradient_computation': gradient_time,
                'structure_tensor': tensor_time,
                'harris_response': response_time,
                'non_maximum_suppression': nms_time,
                'corner_extraction': extraction_time,
                'total': total_time
            }
        }

# 創建自定義Harris檢測器
custom_harris = CustomHarrisDetector(k=0.04, window_size=5, gaussian_sigma=1.2)
print("✅ 自定義Harris角點檢測器初始化完成")

### 🎯 挑戰任務 2.1: 測試自定義Harris算法

**任務**: 測試自定義Harris角點檢測並與OpenCV實現比較。

In [ ]:
def compare_harris_implementations(image_path):
    """
    比較自定義Harris實現與OpenCV官方實現
    
    Args:
        image_path: 測試圖像路徑
    """
    if not os.path.exists(image_path):
        print(f"圖像不存在: {image_path}")
        return
    
    # 載入圖像
    image = load_image(image_path)
    if image is None:
        return
    
    # 調整圖像大小
    image_resized = resize_image(image, max_width=400)
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    
    print(f"\n🔍 Harris角點檢測比較測試: {os.path.basename(image_path)}")
    print(f"圖像大小: {gray.shape}")
    print("-" * 50)
    
    # 測試自定義實現
    print("\n🛠️  自定義實現:")
    custom_result = custom_harris.detect_corners(gray, threshold=0.01, nms_window=5)
    
    # 測試OpenCV實現
    print("\n📚 OpenCV實現:")
    start_time = time.time()
    
    # OpenCV Harris角點檢測
    opencv_response = cv2.cornerHarris(gray, 5, 3, 0.04)
    
    # 應用相同的後處理
    opencv_response_dilated = cv2.dilate(opencv_response, None)
    threshold = 0.01 * opencv_response.max()
    opencv_corners = np.where(opencv_response > threshold)
    opencv_corner_list = list(zip(opencv_corners[1], opencv_corners[0]))
    
    opencv_time = (time.time() - start_time) * 1000
    print(f"  檢測到 {len(opencv_corner_list)} 個角點，處理時間: {opencv_time:.1f}ms")
    
    # 創建可視化圖像
    def draw_corners(img, corners, color=(0, 255, 0), radius=3):
        """在圖像上繪製角點"""
        result = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        for x, y in corners:
            cv2.circle(result, (int(x), int(y)), radius, color, -1)
        return result
    
    # 繪製角點檢測結果
    custom_visualization = draw_corners(gray, custom_result['corners'], (0, 255, 0), 2)
    opencv_visualization = draw_corners(gray, opencv_corner_list, (255, 0, 0), 2)
    
    # 創建響應熱圖
    custom_response_vis = cv2.applyColorMap(
        (custom_result['response'] / np.max(custom_result['response']) * 255).astype(np.uint8),
        cv2.COLORMAP_JET
    )
    
    opencv_response_vis = cv2.applyColorMap(
        (opencv_response / np.max(opencv_response) * 255).astype(np.uint8),
        cv2.COLORMAP_JET
    )
    
    # 顯示中間步驟
    intermediate_images = [
        custom_result['original'],
        custom_result['gradient_x'],
        custom_result['gradient_y'],
        custom_response_vis
    ]
    
    intermediate_titles = [
        "原始圖像",
        "X方向梯度",
        "Y方向梯度",
        "Harris響應"
    ]
    
    print("\n📊 顯示中間處理步驟:")
    display_multiple_images(intermediate_images, intermediate_titles, figsize=(16, 8))
    
    # 顯示最終比較結果
    final_images = [
        cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR),
        custom_visualization,
        opencv_visualization,
        np.hstack([custom_response_vis, opencv_response_vis])
    ]
    
    final_titles = [
        "原始圖像",
        f"自定義Harris\n{len(custom_result['corners'])} 角點, {custom_result['processing_times']['total']:.1f}ms",
        f"OpenCV Harris\n{len(opencv_corner_list)} 角點, {opencv_time:.1f}ms",
        "響應比較\n(左:自定義, 右:OpenCV)"
    ]
    
    print("\n🏆 最終結果比較:")
    display_multiple_images(final_images, final_titles, figsize=(16, 10))
    
    # 性能分析
    print("\n📈 性能分析:")
    print("-" * 40)
    custom_time = custom_result['processing_times']['total']
    print(f"自定義實現:")
    print(f"  角點數量: {len(custom_result['corners'])}")
    print(f"  處理時間: {custom_time:.1f}ms")
    print(f"\nOpenCV實現:")
    print(f"  角點數量: {len(opencv_corner_list)}")
    print(f"  處理時間: {opencv_time:.1f}ms")
    print(f"\n性能比值: {custom_time/opencv_time:.1f}x")
    
    # 詳細時間分解
    print("\n⏱️  自定義實現時間分解:")
    for step, time_ms in custom_result['processing_times'].items():
        if step != 'total':
            percentage = (time_ms / custom_time) * 100
            print(f"  {step:22}: {time_ms:6.1f}ms ({percentage:4.1f}%)")
    
    # 質量評估
    print("\n🎯 算法參數:")
    print(f"  Harris參數k:        {custom_result['parameters']['k']}")
    print(f"  相對閾值:           {custom_result['parameters']['threshold']}")
    print(f"  自適應閾值:         {custom_result['parameters']['adaptive_threshold']:.2f}")
    print(f"  最大響應值:         {custom_result['parameters']['max_response']:.2f}")
    
    return custom_result, {
        'corners': opencv_corner_list,
        'response': opencv_response,
        'time': opencv_time
    }

# 測試自定義Harris實現
harris_test_images = [
    "../../assets/images/basic/face03.jpg",
    "../../assets/images/basic/faces01.jpg"
]

for image_path in harris_test_images:
    if os.path.exists(image_path):
        custom_result, opencv_result = compare_harris_implementations(image_path)
        break
else:
    print("⚠️ 沒有找到測試圖像")

## 挑戰總結與評估

### 🏆 完成成就評估

請根據你的完成情況進行自評：

In [ ]:
class AlgorithmImplementationAssessment:
    """自定義算法實作評估系統"""
    
    def __init__(self):
        """初始化評估系統"""
        self.criteria = {
            'correctness': {
                'name': '算法正確性',
                'weight': 0.3,
                'description': '實現是否符合算法理論'
            },
            'performance': {
                'name': '性能效率',
                'weight': 0.25,
                'description': '與標準實現的速度比較'
            },
            'code_quality': {
                'name': '代碼品質',
                'weight': 0.2,
                'description': '代碼結構、註解、可讀性'
            },
            'optimization': {
                'name': '優化程度',
                'weight': 0.15,
                'description': '使用了哪些優化技術'
            },
            'innovation': {
                'name': '創新改進',
                'weight': 0.1,
                'description': '是否有創新的改進或擴展'
            }
        }
    
    def evaluate_implementation(self, algorithm_name, scores):
        """
        評估算法實現
        
        Args:
            algorithm_name: 算法名稱
            scores: 各項評分字典 (0-10分)
        
        Returns:
            dict: 評估結果
        """
        print(f"\n📊 {algorithm_name} 實現評估")
        print("=" * 50)
        
        total_score = 0
        weighted_scores = {}
        
        for criterion, info in self.criteria.items():
            if criterion in scores:
                score = scores[criterion]
                weighted_score = score * info['weight']
                total_score += weighted_score
                weighted_scores[criterion] = weighted_score
                
                print(f"{info['name']:12}: {score:4.1f}/10 (權重{info['weight']*100:3.0f}%) = {weighted_score:4.2f}")
        
        print("-" * 50)
        print(f"總分: {total_score:.2f}/10")
        
        # 等級評定
        if total_score >= 9.0:
            grade = "專家級 ⭐⭐⭐⭐⭐"
            comment = "卓越的實現，達到生產環境標準"
        elif total_score >= 8.0:
            grade = "高級 ⭐⭐⭐⭐"
            comment = "優秀的實現，具有實用價值"
        elif total_score >= 7.0:
            grade = "中高級 ⭐⭐⭐⭐"
            comment = "良好的實現，理解深入"
        elif total_score >= 6.0:
            grade = "中級 ⭐⭐⭐"
            comment = "基本正確的實現"
        else:
            grade = "入門級 ⭐⭐"
            comment = "需要進一步改進"
        
        print(f"\n🏆 評級: {grade}")
        print(f"💬 評價: {comment}")
        
        return {
            'total_score': total_score,
            'weighted_scores': weighted_scores,
            'grade': grade,
            'comment': comment
        }
    
    def generate_improvement_suggestions(self, scores):
        """
        生成改進建議
        
        Args:
            scores: 評分字典
        """
        print("\n💡 改進建議:")
        print("-" * 30)
        
        suggestions = []
        
        if scores.get('correctness', 10) < 8:
            suggestions.append("• 仔細檢查算法步驟，確保實現符合理論")
            suggestions.append("• 增加更多測試用例驗證正確性")
        
        if scores.get('performance', 10) < 7:
            suggestions.append("• 使用Numba或Cython進行加速優化")
            suggestions.append("• 優化數據結構和循環結構")
            suggestions.append("• 考慮使用向量化操作")
        
        if scores.get('code_quality', 10) < 8:
            suggestions.append("• 添加詳細的函數文檔字符串")
            suggestions.append("• 改進變量命名和代碼結構")
            suggestions.append("• 增加錯誤處理和邊界條件檢查")
        
        if scores.get('optimization', 10) < 7:
            suggestions.append("• 分析算法瓶頸並進行針對性優化")
            suggestions.append("• 考慮內存訪問模式優化")
            suggestions.append("• 實現多線程或GPU加速")
        
        if scores.get('innovation', 10) < 6:
            suggestions.append("• 嘗試改進算法參數或添加自適應機制")
            suggestions.append("• 結合其他算法或技術進行創新")
            suggestions.append("• 針對特定應用場景進行優化")
        
        if not suggestions:
            suggestions.append("• 已經是很優秀的實現！")
            suggestions.append("• 可以考慮將實現貢獻給開源社群")
        
        for suggestion in suggestions:
            print(suggestion)

# 創建評估系統
assessor = AlgorithmImplementationAssessment()

# 示例評估 - 請根據你的實際完成情況調整分數
print("\n🎯 自定義算法實作挑戰 - 成果評估")
print("請根據你的實際完成情況，為以下方面打分 (0-10分):")

# Canny算法評估示例
canny_scores = {
    'correctness': 8.5,    # 算法步驟完整，與OpenCV結果相似度高
    'performance': 6.0,    # 比OpenCV慢約3-5倍，但可接受
    'code_quality': 8.0,   # 代碼結構清晰，註解完整
    'optimization': 7.0,   # 使用了Numba加速
    'innovation': 5.0      # 基本實現，無特殊創新
}

canny_result = assessor.evaluate_implementation("自定義Canny邊緣檢測", canny_scores)
assessor.generate_improvement_suggestions(canny_scores)

# Harris算法評估示例
harris_scores = {
    'correctness': 8.0,    # 角點檢測結果合理
    'performance': 5.5,    # 性能有待優化
    'code_quality': 7.5,   # 代碼質量良好
    'optimization': 6.5,   # 部分使用了加速技術
    'innovation': 4.5      # 標準實現
}

harris_result = assessor.evaluate_implementation("自定義Harris角點檢測", harris_scores)
assessor.generate_improvement_suggestions(harris_scores)

## 🚀 進階挑戰方向

完成基本實現後，你可以嘗試以下進階挑戰：

### 1. 性能極致優化
- **SIMD指令優化**: 使用NumPy的向量化操作
- **記憶體優化**: 減少不必要的記憶體分配
- **並行處理**: 使用多線程或多處理
- **GPU加速**: 使用CuPy或OpenCV的CUDA功能

### 2. 算法創新改進
- **自適應參數**: 根據圖像內容自動調整參數
- **多尺度處理**: 實現金字塔結構的多尺度檢測
- **魯棒性增強**: 對噪聲和光照變化的適應性
- **混合算法**: 結合多種算法的優點

### 3. 應用場景特化
- **實時處理**: 針對影片流的優化
- **嵌入式優化**: 適合移動設備的輕量化
- **特定領域**: 醫學影像、遙感影像等專用版本

### 4. 完整系統整合
- **管道化處理**: 建立完整的視覺處理管道
- **可視化界面**: 添加參數調整的GUI
- **效能基準測試**: 建立標準化的測試框架

## 📚 學習資源推薦

### 經典論文
- **Canny Edge Detection**: "A Computational Approach to Edge Detection" - John Canny (1986)
- **Harris Corner Detection**: "A Combined Corner and Edge Detector" - Harris & Stephens (1988)
- **SIFT**: "Distinctive Image Features from Scale-Invariant Keypoints" - David Lowe (2004)

### 實現參考
- OpenCV 源代碼研究
- scikit-image 實現分析
- 學術界開源實現

### 優化技術
- Intel IPP (Integrated Performance Primitives)
- NVIDIA NPP (NVIDIA Performance Primitives)
- OpenMP 並行編程

## 🎯 挑戰完成檢核表

### 基礎要求 ⭐⭐⭐
- [ ] 完成至少一個完整算法的從零實現
- [ ] 實現結果與標準庫基本一致
- [ ] 代碼結構清晰，有適當註解

### 進階要求 ⭐⭐⭐⭐
- [ ] 實現多個算法並進行比較
- [ ] 使用了性能優化技術
- [ ] 建立了完整的測試和驗證流程

### 專家要求 ⭐⭐⭐⭐⭐
- [ ] 性能接近或超越標準實現
- [ ] 有創新的改進或應用
- [ ] 代碼質量達到生產環境標準

**完成基礎要求即可進入下一個高級練習！** 🎉

---

### 💬 反思與總結

通過這個挑戰，你應該已經：
1. **深入理解**了經典電腦視覺算法的數學原理
2. **掌握了**從理論到實作的完整流程
3. **學會了**性能優化和代碼質量控制
4. **建立了**算法評估和比較的方法論

這些技能將為你在電腦視覺領域的深入研究和工程開發奠定堅實基礎！